The scheduler class defines a generic interface to scheduling events.

In [5]:
import sched, time
import pickle
from PIL import Image
import cv2
import os

def extract_hist(img):
    color = ('b','g','r')
    histogram = np.empty([3,256])
    for channel,col in enumerate(color):
        histr = cv2.calcHist([np.asarray(img)],[channel],None,[256],[0,256])
        histogram[channel] = np.transpose(histr)
    return histogram.ravel()


loaded_model = pickle.load(open('./Classifiers/GaussianNB', 'rb'))


Here, we chose to use this method:
    scheduler.enter(delay, priority, action, argument=(), kwargs={})
    
This method execute the 'action' function every delay time.

In our case, we iterate over the images folder to look for bad images and if the classifier tells that the image is not good, it will be deleted.

In [1]:
s = sched.scheduler(time.time, time.sleep)

def check_images(sc): 
    img_dir = "./ReceivedImages/"
    img_paths = [join(img_dir,filename) for filename in os.listdir(img_dir)]
    
    for image in img_paths:
        isGood = loaded_model.predict([extract_hist(Image.open( image ))])
        if(!isGood):
            os.remove(image)
            print(f'image {os.path.basename(image)} deleted')
            
    s.enter(3600, 1, check_images, (sc,))
    

s.enter(1, 1, check_images,(s,))
s.run()